# TP 3 : Text segmentation using Hidden Markov Models <br>BEJAOUI AHMED - MEJRI AYMEN

### Question 1  : Give the value of the π vector of the initial probabilities¶

Etant donnée que l'état initial de départ est l'entête nous constatons donc que le vecteur d'initialisation  : $$\pi=[1,0]$$

### Question 2 : What is the probability to move from state 1 to state 2 ? What is the probability to remain in state 2 ? What is the lower/higher probability ? Try to explain why

$$ A = \begin{bmatrix} 
0.999218078035812 & 0.000781921964187974 \\
0 & 1
\end{bmatrix}$$

Disposant de la matrice A , nous pouvons conclure que la probabilité de  passer de l'état 1 vers l'état 2 correspond à $A_{12}=0.000781921964187974$  et la probabilité de rester dans l'état 2 est  $A_{22}=1$
<br>
la plus faible probabilité est est p(2|1) = 0 ce qui est tout à fait prévisible parce que nous ne pouvons pas passer à l'état 1 header lorsque nous nous trouvons à l'état 2 (corps du mail) .
<br>
La plus grande probabilité est p(2|2) = 1 c'est à dire lorsque nous sommes dans l'état 2 (corps du mail), nous y restons 


### Question 3 : What is the size of the corresponding matrix ?

La taille de la matrice est 256x2 . En effet nous avons 256 caractères possibles dans le code ASCII donc nous avons 256 lignes dans la matrice et nous disposons seulement de 2 états possible qui sont s=1 et s=2 donc nous avons 2 colonnes .

### Question 4 : Print the track and present and discuss the results obtained on mail11.txt to mail30.txt

In [1]:
from glob import glob
import os.path as op
import numpy as np

filenames = sorted(glob(op.join('dat','*.dat')))
texts_pred = [np.loadtxt(f,dtype=int) for f in filenames]
len(texts_pred)

30

#### Initialisation :

In [2]:
A=np.zeros((2,2))
A[0,0]=0.999218078035812; A[0,1]=0.000781921964187974 ; A[1,0]=0 ; A[1,1]=1
B = np.loadtxt('P.text', dtype=float)
pi=[1,0]
q=[0,1]

#### La Fonction Viterbi :

In [3]:
def Viterbi_function(A,B,pi,O,q):
    """
    Input : 
    
    A : Matrice de transition des différents états
    B : Matrice de transition des observations sachant les états
    pi: vecteur d'initialisation 
    O : vecteur des observations 
    q : Vecteur des différents états possibles
    
    Output:
    
    qhat : la séquence d'état prédite.
    
    """
    
    dl=np.zeros((len(q),len(O)))
    phi=np.zeros((len(q),len(O)))
    
    inter=0
    qhT=0
    
    for i in range(len(q)):
        dl[i,0]=B[O[0],i]*pi[i]
    
    for t in range(len(O)-1):
        for j in range(len(q)):
            inter=np.max([A[i,j]*dl[i,t] for i in range(len(q))])
            dl[j,t+1]=inter*B[O[t+1],j]
            phi[j,t+1]=np.argmax([A[i,j]*dl[i,t] for i in range(len(q))])
        dl[:,t+1] = dl[:,t+1]*(10**(-1*(int(np.log10(np.max(dl[:,t+1])))+1)))
    qh=np.array([])
    qhT=np.argmax([dl[j,len(O)-1] for j in range(len(q))])
    qh=np.append(qhT,qh)
    for i in range (len(O)-1,0,-1):
        qhT=int(qhT)
        qhT=phi[qhT,i]
        qh=np.append(qhT,qh)
        qh=np.array([int(i) for i in qh])
    return qh

#### Etude de la performance de l'algorithme:

In [4]:
filenames = sorted(glob(op.join( 'dat', '*.txt')))
train = [len(open(f).read()) for f in filenames if f.rstrip().endswith("h.txt")]
texts = [np.loadtxt(f.rstrip().replace("h.txt",".dat"),dtype="int") for f in filenames if f.rstrip().endswith("h.txt")]
state_pred=np.array([])
for text in texts:
    state_pred=np.append(state_pred,np.bincount(Viterbi_function(A,B,pi,text,q))[0])

score=np.array([])
score=1-np.abs(train-state_pred)/state_pred

In [5]:
states_pred=np.array([])
for text in texts_pred:
    states_pred=np.append(states_pred,np.bincount(Viterbi_function(A,B,pi,text,q))[0])

In [6]:
print('L\'algorithme a une précision de : ',score.mean())

L'algorithme a une précision de :  0.983381544512


#### initialisation sur un ensemble de texte de mail11 jusqu'à mail30 : 

nous prenons ici les deux fichiers mail11.txt et mail30.txt pour ces deux fichiers nous avons respectivement 2850 et 2249 caractères dans le header. nous avons déterminer ces résultat en regarder le texte avec wordPad.


In [7]:
train=[2850,2249]
state_pred=np.array([])
O1= np.loadtxt('dat/mail11.dat',dtype=int)
O2= np.loadtxt('dat/mail30.dat',dtype=int)
state_pred=np.append(state_pred,np.bincount(Viterbi_function(A,B,pi,O1,q))[0])
state_pred=np.append(state_pred,np.bincount(Viterbi_function(A,B,pi,O2,q))[0])
score=np.array([])
score=1-np.abs(train-state_pred)/state_pred

In [8]:
print('L\'algorithme a une précision de : ',score.mean())

L'algorithme a une précision de :  0.982337278255


#### Visualisation sur un texte : 

Pour la visualisation du texte nous avons eu un problème avec la commande perl. Nous avons donc choisi de développer notre propre fonction.

In [9]:
def segment(path,file_name,size=None):
    text=None
    with open(file_name) as f:
        text=f.read()
    n =np.argmax(path)
    a=0
    b=len(text)
    if size != None:
        a=max(0,n-size)
        b=min(len(text),n+size)
    print(text[a:n]+ '\n================================== coupez ici ==============================\n'+text[n:b])
    return None

In [10]:
O2= np.loadtxt('dat/mail13.dat',dtype=int)
segment(Viterbi_function(A,B,pi,O2,q),'dat/mail13.txt')

From ilug-admin@linux.ie  Thu Aug 22 16:27:21 2002
Return-Path: <ilug-admin@linux.ie>
Delivered-To: zzzz@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 7A28A43F99
	for <zzzz@localhost>; Thu, 22 Aug 2002 11:27:21 -0400 (EDT)
Received: from phobos [127.0.0.1]
	by localhost with IMAP (fetchmail-5.9.0)
	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 16:27:21 +0100 (IST)
Received: from lugh.tuatha.org (root@lugh.tuatha.org [194.125.145.45]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MFQmZ12280 for
    <zzzz-ilug@spamassassin.taint.org>; Thu, 22 Aug 2002 16:26:48 +0100
Received: from lugh (root@localhost [127.0.0.1]) by lugh.tuatha.org
    (8.9.3/8.9.3) with ESMTP id QAA07188; Thu, 22 Aug 2002 16:25:32 +0100
Received: from moe.jinny.ie ([193.120.171.3]) by lugh.tuatha.org
    (8.9.3/8.9.3) with ESMTP id QAA07145 for <ilug@linux.ie>; Thu,
    22 Aug 2002 16:25:24 +0100
X-Authentication-Warnin

### Question 5 : How would you model the problem if you had to segment the mails in more than two parts (for example : header, body, signature) ?

On utilisera dans ce cas aussi le model HMM mais l'entrée de l'algorithme est différente : l'espace d'état sera de taille 3 qui prendra les valeurs suivantes : header(=0),body(=1),et signature(=2) et le vecteur d'initialisation est de dimention 3 qui commence toujours par le header et par conséquent $\pi = (1,0,0)$ 

Concernant la matrice de transition A, elle est de taille 3x3 et telle que $A_{13}=0$ , $A_{21}=0$,$A_{13}=0$ et $A_{23}=0$

$$ A = \begin{bmatrix} 
A_{11} & A_{12} & 0 \\
0 & A_{22} & A_{23} \\
0 & 0 & 1
\end{bmatrix}$$

et la matrice B  qui représente la matrice des probabilités $P(c|s)$ devient une matrice de taille 256x3 .

### Question 6 : How would you model the problem of separating the portions of mail included, knowing that they always start with the character ">".

Pour cette question , nous aurons 4 etats possible qui sont : header,body_included ,body(le texte) et la signature . le vecteur d'initailisation $\pi$ est de taille 4x1 : $$ \pi=[1, 0, 0, 0]$$ 
La matrice de transition A est de dimension 4x4 :
$$ A = \begin{bmatrix} 
A_{11} & A_{12} & A_{12} & 0\\
0 & A_{22} & A_{23} & A_{24}\\
0 & A_{32} & A_{33} & A_{34}\\
0 & 0 & 0 & 1
\end{bmatrix}$$

la probabilité du caractère appartenant à l'état 3 est plus élevée par rapport aux autres états - cette information doit être incluse dans la matrice de probabilité conditionnelle B
<br> La matrice B sera de taille 256x4.